In [1]:
# Install necessary packages
!pip install --user tensorflow
!pip install --user xgboost
!pip install --user opencv-python-headless
!pip install --user imgaug
!pip install --user scikit-image

# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import warnings

from PIL import Image
from sklearn.cluster import DBSCAN
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split  # Added train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    Input, BatchNormalization, Activation, concatenate
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

try:
    from skimage import feature
except ModuleNotFoundError:
    print("scikit-image module not found. Please install it using '!pip install --user scikit-image'")

# Rest of your code continues...

# Rest of your code continues...


# Set logging
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore')

# Constants
IMAGE_DIR = './documents'
PREPROCESSED_DIR = './documents/preprocessed_images'
EXCEL_FILE = 'Meta_pic_3.xlsx'
LEFT, TOP, IMG_WIDTH, IMG_HEIGHT = 232, 60, 495, 475
RIGHT, BOTTOM = LEFT + IMG_WIDTH, TOP + IMG_HEIGHT
CROP_TOP, CROP_HEIGHT, CROP_LEFT, CROP_WIDTH = 290, 100, 215, 125
CROP_BOTTOM, CROP_RIGHT = CROP_TOP + CROP_HEIGHT, CROP_LEFT + CROP_WIDTH
MAX_FEATURES, GOOD_MATCH_PERCENT, MAX_DELTA = 500, 0.15, 200
MIN_DELTA_X, MAX_DELTA_X = -CROP_LEFT, IMG_WIDTH - CROP_RIGHT
MIN_DELTA_Y, MAX_DELTA_Y = -CROP_TOP, IMG_HEIGHT - CROP_BOTTOM
NUM_FOLDS = 5  # For K-fold cross-validation

# Create directories
os.makedirs(PREPROCESSED_DIR, exist_ok=True)

# Configure TensorFlow to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logging.info("GPU memory growth set")
    except RuntimeError as e:
        logging.warning(e)

# Function to load and preprocess images
def load_and_preprocess_images(image_dir):
    image_files = [file for file in os.listdir(image_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]
    images = []
    for file in image_files:
        img_path = os.path.join(image_dir, file)
        img = Image.open(img_path).convert('L')
        img_cropped = img.crop((LEFT, TOP, RIGHT, BOTTOM))
        images.append((file, np.array(img_cropped)))
    return images

# Function for image alignment using ORB
def align_images(reference_image, images):
    aligned_images = []
    offsets = []
    orb = cv2.ORB_create(MAX_FEATURES)
    ref_kp, ref_des = orb.detectAndCompute(reference_image, None)
    for filename, img in images:
        try:
            kp, des = orb.detectAndCompute(img, None)
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(ref_des, des)
            matches = sorted(matches, key=lambda x: x.distance)
            matches = matches[:int(len(matches) * GOOD_MATCH_PERCENT)]
            points1 = np.float32([ref_kp[m.queryIdx].pt for m in matches])
            points2 = np.float32([kp[m.trainIdx].pt for m in matches])
            deltas = points2 - points1
            # Filter deltas
            deltas = deltas[(deltas[:, 0] > MIN_DELTA_X) & (deltas[:, 0] < MAX_DELTA_X) &
                            (deltas[:, 1] > MIN_DELTA_Y) & (deltas[:, 1] < MAX_DELTA_Y)]
            if len(deltas) == 0:
                offset = (0, 0)
            else:
                # Clustering to find the most consistent offset
                clustering = DBSCAN(eps=10, min_samples=5).fit(deltas)
                labels, counts = np.unique(clustering.labels_, return_counts=True)
                if len(labels) > 1:
                    max_label = labels[np.argmax(counts[labels != -1])]
                    offset = deltas[clustering.labels_ == max_label].mean(axis=0)
                else:
                    offset = deltas.mean(axis=0)
            offsets.append(offset)
            aligned_images.append((filename, img))
        except Exception as e:
            logging.warning(f"Alignment failed for {filename}: {e}")
            offsets.append((0, 0))
            aligned_images.append((filename, img))
    return aligned_images, offsets

# Function to preprocess images (cropping and saving)
def preprocess_and_save_images(images, offsets, preprocessed_dir):
    os.makedirs(preprocessed_dir, exist_ok=True)
    for (filename, img), offset in zip(images, offsets):
        offset_left, offset_top = offset
        cropped_top = int(CROP_TOP + offset_top)
        cropped_bottom = int(CROP_BOTTOM + offset_top)
        cropped_left = int(CROP_LEFT + offset_left)
        cropped_right = int(CROP_RIGHT + offset_left)
        # Ensure the cropping is within image bounds
        if (0 <= cropped_top < cropped_bottom <= img.shape[0]) and \
           (0 <= cropped_left < cropped_right <= img.shape[1]):
            cropped_img = img[cropped_top:cropped_bottom, cropped_left:cropped_right]
            preprocessed_path = os.path.join(preprocessed_dir, filename)
            Image.fromarray(cropped_img).save(preprocessed_path)
        else:
            logging.warning(f"Cropping out of bounds for {filename}")

# Function to apply image augmentation
def augment_images(images, labels):
    datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    augmented_images = []
    augmented_labels = []
    for img, label in zip(images, labels):
        img = img.reshape((1,) + img.shape)
        aug_iter = datagen.flow(img, batch_size=1)
        for _ in range(3):  # Generate 3 augmented images per original image
            aug_img = next(aug_iter)[0].astype('float32')
            augmented_images.append(aug_img)
            augmented_labels.append(label)
    return np.array(augmented_images), np.array(augmented_labels)

# Function to load labels and match with images
def load_labels_and_images(excel_file, preprocessed_dir):
    df = pd.read_excel(excel_file)
    df['filename'] = df['record_id'].astype(str) + '_image_data_' + df['redcap_repeat_instance'].astype(str) + '_raw_image.jpg'
    df['onsd'] = df['onsd'].astype(str).str.replace(',', '.').astype(float)
    images = []
    labels = []
    for _, row in df.iterrows():
        image_path = os.path.join(preprocessed_dir, row['filename'])
        if os.path.exists(image_path):
            img = Image.open(image_path).resize((128, 128))
            images.append(np.array(img))
            labels.append(row['onsd'])
        else:
            logging.warning(f"Preprocessed image not found: {row['filename']}")
    return np.array(images), np.array(labels)

# Function to build UNet model
def build_unet_model(input_shape=(128, 128, 1)):
    inputs = Input(shape=input_shape)
    # Encoder
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Decoder
    u3 = tf.keras.layers.UpSampling2D((2, 2))(c2)
    u3 = concatenate([u3, c1])
    c3 = Conv2D(16, (3, 3), activation='relu', padding='same')(u3)
    c3 = BatchNormalization()(c3)
    c3 = Conv2D(16, (3, 3), activation='relu', padding='same')(c3)

    outputs = Conv2D(1, (1, 1), activation='linear')(c3)
    outputs = Flatten()(outputs)
    outputs = Dense(1)(outputs)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Function to perform K-fold cross-validation
def cross_validate_model(model_fn, X, y, n_splits=NUM_FOLDS, epochs=25, batch_size=32):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_no = 1
    all_scores = []
    for train_index, val_index in kfold.split(X, y):
        logging.info(f'Training fold {fold_no}')
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        model = model_fn()
        model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
        # Early stopping and model checkpoint
        callbacks = [
            EarlyStopping(patience=5, restore_best_weights=True),
            ModelCheckpoint(f'model_fold_{fold_no}.h5', save_best_only=True)
        ]
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                  validation_data=(X_val, y_val), callbacks=callbacks, verbose=0)
        scores = model.evaluate(X_val, y_val, verbose=0)
        logging.info(f'Fold {fold_no} MAE: {scores[1]}')
        all_scores.append(scores[1])
        fold_no += 1
    avg_score = np.mean(all_scores)
    logging.info(f'Average MAE over {n_splits} folds: {avg_score}')
    return avg_score

# Function to build and train ensemble models
def train_ensemble_models(X_train_flat, y_train):
    estimators = [
        ('rf', RandomForestRegressor(random_state=42)),
        ('xgb', XGBRegressor(random_state=42))
    ]
    # Stacking Regressor
    stack_model = StackingRegressor(
        estimators=estimators,
        final_estimator=LinearRegression(),
        cv=NUM_FOLDS,
        n_jobs=-1
    )
    stack_model.fit(X_train_flat, y_train)
    return stack_model

# Function to perform hyperparameter tuning
def tune_hyperparameters(X_train_flat, y_train):
    param_grid = {
        'max_depth': [3, 5, 7],
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'subsample': [0.8, 1],
    }
    xgb = XGBRegressor(random_state=42)
    grid_search = GridSearchCV(xgb, param_grid, scoring='neg_mean_absolute_error', cv=NUM_FOLDS, n_jobs=-1)
    grid_search.fit(X_train_flat, y_train)
    logging.info(f'Best parameters found: {grid_search.best_params_}')
    return grid_search.best_estimator_

# Main execution flow
if __name__ == '__main__':
    # Load and preprocess images
    logging.info("Loading and preprocessing images...")
    raw_images = load_and_preprocess_images(IMAGE_DIR)
    reference_image = raw_images[0][1]
    aligned_images, offsets = align_images(reference_image, raw_images)
    preprocess_and_save_images(aligned_images, offsets, PREPROCESSED_DIR)

    # Load labels and images
    logging.info("Loading labels and images...")
    images, labels = load_labels_and_images(EXCEL_FILE, PREPROCESSED_DIR)

    # Apply edge detection as additional preprocessing
    logging.info("Applying edge detection...")
    images_edges = np.array([feature.canny(img.astype('float32') / 255.0) for img in images])
    images_edges = images_edges[..., np.newaxis]

    # Normalize images
    images = images.astype('float32') / 255.0
    images = images[..., np.newaxis]

    # Augment images
    logging.info("Augmenting images...")
    images_augmented, labels_augmented = augment_images(images, labels)
    images_combined = np.vstack((images, images_augmented))
    labels_combined = np.hstack((labels, labels_augmented))

    # Split data
    logging.info("Splitting data...")
    X_train, X_test, y_train, y_test = train_test_split(images_combined, labels_combined, test_size=0.2, random_state=42)

    # Build and cross-validate UNet model
    logging.info("Building and cross-validating UNet model...")
    unet_mae = cross_validate_model(build_unet_model, X_train, y_train)

    # Prepare data for traditional ML models
    logging.info("Preparing data for traditional ML models...")
    X_train_flat = X_train.reshape(X_train.shape[0], -1)
    X_test_flat = X_test.reshape(X_test.shape[0], -1)

    # Hyperparameter tuning
    logging.info("Hyperparameter tuning for XGBoost...")
    xgb_best = tune_hyperparameters(X_train_flat, y_train)

    # Train ensemble model
    logging.info("Training ensemble model...")
    stack_model = train_ensemble_models(X_train_flat, y_train)

    # Evaluate models
    logging.info("Evaluating models...")
    # UNet evaluation
    unet_model = build_unet_model()
    unet_model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
    unet_model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test), verbose=0)
    unet_predictions = unet_model.predict(X_test).flatten()
    unet_mae = mean_absolute_error(y_test, unet_predictions)
    unet_rmse = np.sqrt(mean_squared_error(y_test, unet_predictions))

    # XGBoost evaluation
    xgb_predictions = xgb_best.predict(X_test_flat)
    xgb_mae = mean_absolute_error(y_test, xgb_predictions)
    xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))

    # Ensemble evaluation
    stack_predictions = stack_model.predict(X_test_flat)
    stack_mae = mean_absolute_error(y_test, stack_predictions)
    stack_rmse = np.sqrt(mean_squared_error(y_test, stack_predictions))

    # Compile results
    logging.info("Compiling results...")
    comparison_df = pd.DataFrame({
        'Model': ['UNet', 'XGBoost (Tuned)', 'Ensemble (Stacked)'],
        'MAE': [unet_mae, xgb_mae, stack_mae],
        'RMSE': [unet_rmse, xgb_rmse, stack_rmse]
    })

    # Print comparison results
    print(comparison_df)

    # Save comparison results
    comparison_df.to_csv('optimized_model_comparison.csv', index=False)

    # Plot MAE comparison
    plt.figure(figsize=(10, 6))
    plt.bar(comparison_df['Model'], comparison_df['MAE'], color='skyblue')
    plt.xlabel('Model')
    plt.ylabel('Mean Absolute Error (MAE)')
    plt.title('Model Comparison - MAE')
    plt.savefig('optimized_model_mae_comparison.png')
    plt.show()

    # Bland-Altman Plot for UNet
    def bland_altman_plot(actual, predicted, model_name):
        difference = actual - predicted
        avg = (actual + predicted) / 2
        mean_diff = np.mean(difference)
        std_diff = np.std(difference)
        upper_limit = mean_diff + 1.96 * std_diff
        lower_limit = mean_diff - 1.96 * std_diff

        plt.figure(figsize=(10, 6))
        plt.scatter(avg, difference, alpha=0.5)
        plt.axhline(mean_diff, color='red', linestyle='--', label='Mean Difference')
        plt.axhline(upper_limit, color='gray', linestyle='--', label='Upper Limit')
        plt.axhline(lower_limit, color='gray', linestyle='--', label='Lower Limit')
        plt.xlabel('Average of Actual and Predicted')
        plt.ylabel('Difference between Actual and Predicted')
        plt.title(f'Bland-Altman Plot - {model_name}')
        plt.legend()
        plt.savefig(f'bland_altman_{model_name.lower().replace(" ", "_")}.png')
        plt.show()

    bland_altman_plot(y_test, unet_predictions, 'UNet')


ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.

[notice] 

scikit-image module not found. Please install it using '!pip install --user scikit-image'


INFO:root:Loading labels and images...
INFO:root:Applying edge detection...


NameError: name 'feature' is not defined